In [79]:
import pandas as pd
import numpy as np
import json
import cv2

ModuleNotFoundError: No module named 'cv2'

In [ ]:

with open(r"\Users\Royal_Tech\Documents\BIOIRC\arcade\syntax\test\annotations\test.json", "r") as f:
    data = json.load(f)


How big are the top level fields

In [21]:
print(len(data["categories"]))
print(len(data["annotations"]))
print(len(data["images"]))

26
1672
300


Insight into their structure

In [83]:
print(data["categories"])
print(data["annotations"][0])
print(data["images"][0])

[{'id': 1, 'name': '1', 'supercategory': ''}, {'id': 2, 'name': '2', 'supercategory': ''}, {'id': 3, 'name': '3', 'supercategory': ''}, {'id': 4, 'name': '4', 'supercategory': ''}, {'id': 5, 'name': '5', 'supercategory': ''}, {'id': 6, 'name': '6', 'supercategory': ''}, {'id': 7, 'name': '7', 'supercategory': ''}, {'id': 8, 'name': '8', 'supercategory': ''}, {'id': 9, 'name': '9', 'supercategory': ''}, {'id': 10, 'name': '9a', 'supercategory': ''}, {'id': 11, 'name': '10', 'supercategory': ''}, {'id': 12, 'name': '10a', 'supercategory': ''}, {'id': 13, 'name': '11', 'supercategory': ''}, {'id': 14, 'name': '12', 'supercategory': ''}, {'id': 15, 'name': '12a', 'supercategory': ''}, {'id': 16, 'name': '13', 'supercategory': ''}, {'id': 17, 'name': '14', 'supercategory': ''}, {'id': 18, 'name': '14a', 'supercategory': ''}, {'id': 19, 'name': '15', 'supercategory': ''}, {'id': 20, 'name': '16', 'supercategory': ''}, {'id': 21, 'name': '16a', 'supercategory': ''}, {'id': 22, 'name': '16b', 

In [84]:
for category in data["categories"]:
    print(category["name"])


1
2
3
4
5
6
7
8
9
9a
10
10a
11
12
12a
13
14
14a
15
16
16a
16b
16c
12b
14b
stenosis


In [85]:
name_to_color = {
    # RCA
    '1': (255, 0, 0),
    '2': (220, 20, 60),
    '3': (178, 34, 34),
    '4': (139, 0, 0),

    # Posterolateral from RCA
    '16':  (255, 99, 71),
    '16a': (255, 69, 0),
    '16b': (205, 92, 92),
    '16c': (240, 128, 128),

    # Left Main + LAD
    '5': (255, 215, 0),       # Left Main (golden yellow)
    '6': (0, 255, 0),         # LAD proximal
    '7': (34, 139, 34),       # LAD mid
    '8': (0, 128, 0),         # LAD apical

    # Diagonals
    '9':  (0, 191, 255),
    '9a': (30, 144, 255),
    '10': (70, 130, 180),
    '10a':(100, 149, 237),

    # Circumflex / Marginals
    '11': (148, 0, 211),      # Proximal circumflex
    '12': (186, 85, 211),
    '12a':(147, 112, 219),
    '12b':(138, 43, 226),
    '13': (153, 50, 204),

    # Left Posterolaterals + Posterior descending
    '14':  (255, 140, 0),
    '14a': (255, 165, 0),
    '14b': (255, 180, 100),
    '15':  (255, 215, 160),

    # Special case: stenosis
    'stenosis': (255, 255, 255)
}


In [55]:
images_df = pd.DataFrame(data["images"])
categories_df = pd.DataFrame(data["categories"])
annotations_df = pd.DataFrame(data["annotations"])

In [56]:
print(images_df.info())
print(categories_df.info())
print(annotations_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             300 non-null    int64 
 1   width          300 non-null    int64 
 2   height         300 non-null    int64 
 3   file_name      300 non-null    object
 4   license        300 non-null    int64 
 5   flickr_url     300 non-null    object
 6   coco_url       300 non-null    object
 7   date_captured  300 non-null    int64 
dtypes: int64(5), object(3)
memory usage: 18.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             26 non-null     int64 
 1   name           26 non-null     object
 2   supercategory  26 non-null     object
dtypes: int64(1), object(2)
memory usage: 756.0+ bytes
None
<class 'pandas.core.frame.DataFra

In [47]:
sample_id = 1
segmentations = annotations_df[annotations_df["image_id"]==sample_id]

print(segmentations)


   id  image_id  category_id  \
0   1         1           18   
1   2         1            5   
2   3         1           17   
3   4         1           25   
4   5         1           13   
5   6         1           16   

                                        segmentation    area  \
0  [[252.3, 288.21, 257.7, 289.21, 261.5, 291.81,...  1373.0   
1  [[151.59, 207.74, 153.21, 206.12, 154.99, 204....   956.0   
2  [[254.1, 259.41, 257.5, 259.41, 260.1, 260.01,...   428.0   
3  [[223.6, 393.7, 229.18, 395.39, 231.0, 396.6, ...  2270.0   
4  [[188.8, 156.5, 195.9, 156.2, 201.1, 155.6, 20...  1015.0   
5  [[244.42, 210.1, 243.1, 219.4, 243.6, 227.8, 2...  1888.0   

                             bbox  iscrowd           attributes  
0   [252.3, 288.21, 169.6, 102.8]        0  {'occluded': False}  
1   [142.74, 157.1, 49.58, 50.64]        0  {'occluded': False}  
2   [253.54, 259.41, 56.76, 65.0]        0  {'occluded': False}  
3  [218.08, 354.2, 234.82, 79.26]        0  {'occluded': False

In [ ]:
def add_layer(masked_image, verticies, label):
    
    cv2.fillPoly(masked_image, pts=[verticies], label)


In [ ]:
masked_image = np.zeros((512,512))

for index, row in segmentations.iterrows():
    category_id = row["category_id"]
    segmentation_verticies = row["segmentation"]
    segmentation_label = categories_df[categories_df["id"]==category_id]["name"].iloc[0]
    bbox = row["bbox"]

    print("Label fo segmentation")
    print(categories_df[categories_df["id"]==category_id]["name"].iloc[0])
    print("List of verticies")
    print(segmentation_verticies)
    print("Border box coordinates")
    print(bbox)

    add_layer(masked_image,segmentation_verticies,segmentation_label)
    

Label fo segmentation
14a
List of verticies
[[252.3, 288.21, 257.7, 289.21, 261.5, 291.81, 264.5, 293.01, 266.7, 296.81, 267.7, 300.81, 267.9, 303.61, 266.7, 306.21, 264.7, 308.01, 265.5, 310.61, 266.3, 313.61, 261.9, 317.81, 259.9, 323.01, 259.5, 328.41, 259.5, 329.41, 260.1, 336.81, 262.1, 340.01, 262.1, 343.61, 263.1, 349.81, 264.1, 353.81, 265.9, 357.41, 267.1, 360.21, 270.9, 364.21, 272.3, 365.81, 273.5, 366.21, 275.3, 366.61, 278.3, 365.41, 282.9, 365.21, 287.1, 365.41, 287.5, 365.61, 288.1, 366.01, 289.3, 366.81, 290.0, 369.4, 292.5, 372.2, 295.5, 373.81, 298.1, 375.21, 301.1, 376.01, 305.7, 377.81, 308.7, 379.21, 311.1, 379.61, 312.9, 379.61, 315.1, 377.81, 316.7, 375.61, 318.7, 373.21, 321.5, 372.21, 324.3, 371.41, 328.3, 370.61, 331.1, 370.41, 335.1, 372.61, 337.7, 375.41, 338.3, 379.61, 338.5, 381.61, 339.5, 384.41, 341.9, 383.61, 342.6, 379.8, 344.7, 379.81, 347.1, 380.01, 348.1, 383.01, 350.9, 387.81, 356.5, 385.81, 357.3, 385.41, 361.3, 384.21, 363.7, 380.21, 365.9, 378.8